In [173]:
import pytesseract as pyt
import cv2
import os
import spacy
import pandas as pd
from sklearn.model_selection import ParameterGrid

In [174]:
cv2.__version__

'4.6.0'

In [150]:
img_file = 'D:\\Workspaces\\malexandersalazar\\tools-python-image-to-text\\img\\002.jpg'
lang = 'en'

In [159]:
pyt.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
gpu = spacy.prefer_gpu()

In [152]:
text_scores = []

In [153]:
def get_grayscaled(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def get_thresholded(image, block_size, c):
    return cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, block_size, c)

In [154]:
def process(img_file, lang, block_size, c):
    current_img_dir = os.path.dirname(img_file)

    file_name = os.path.basename(img_file)
    file_name_without_ext = os.path.splitext(file_name)[0]
    new_dir = current_img_dir + '\\ocr_' + file_name_without_ext

    if not os.path.exists(new_dir):
        os.mkdir(new_dir)

    img = cv2.imread(img_file)

    thresholded_img = get_thresholded(get_grayscaled(img),block_size,c)

    new_file_name = new_dir + '\\' + file_name_without_ext + '_' + str(block_size) + '_' + str(c)
    cv2.imwrite(new_file_name + '.jpg', thresholded_img)

    if(lang == 'es'):
        tess_lang = 'spa'
    elif(lang == 'en'):
        tess_lang = 'eng'
    else:
        raise Exception('Not supported language')

    return pyt.image_to_string(thresholded_img, tess_lang)


In [155]:
params = {
    'blockSize': [13,21,55,89,233],
    'C': [2,3,5,8,13,21],
}

if(lang == 'es'):
    nlp = spacy.load("es_core_news_md")
elif(lang == 'en'):
    nlp = spacy.load("en_core_web_md")
else:
    raise Exception('Not supported language')

for config in ParameterGrid(params):
    block_size = config['blockSize']
    c = config['C']

    text_result = process(img_file,lang,block_size,c)

    tokens = nlp(text_result)
    df = pd.DataFrame([(w.text, w.pos_) for w in tokens],columns=['text','word_type'])

    nouns = df[df['word_type']=='NOUN']
    verbs = df[df['word_type']=='VERB']
    propernames = df[df['word_type']=='PROPN']
    adjetives = df[df['word_type']=='ADJ']
    prepositions = df[df['word_type']=='ADP']
    verbs = df[df['word_type']=='VERB']
    pronouns = df[df['word_type']=='PRON']
    conjunctions = df[df['word_type']=='CCONJ']

    text_score = len(nouns) + len(verbs) + len(propernames) + len(adjetives) + len(prepositions) + len(verbs) + len(pronouns) + len(conjunctions)
    text_scores.append((text_result,text_score))

In [156]:
scores_df = pd.DataFrame(text_scores, columns=['text_result','text_score'])

In [157]:
text_final_result = scores_df.sort_values('text_score', ascending=False).reset_index(drop=True)['text_result'][0]
print(text_final_result)

Hard reality #1: Despite the rapid growth of renewables, fossi
fuels still provide more than 84 percent of Primary energy,
Renewables (wind, solar, and biofuels) are growing fast bur
still provide just 5 percent, with hydro providing 6.4 percent
and nuclear 4.3 percent,’ At the same time, about 770 million
people in the world have no access to electricity, and 2.6 billion

don’t have access to energy for clean cooking.'4The chal lengg
——— energy for lean cooking. ee
is to Increase access to energy while also decarbonizing it,
“Ceasing Fossil fuel use before there are other options to replace
this 84 percent of global energy use brings real risks of coo-
tinuing or worsening energy poverty. This is often recognized

by climate advocates—for example, when 432 environmen”

tal groups from 53 countries sent a letter to officials in the
Biden Adyinjerenes cc cue - fear che

a


